In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [22]:
#Get CSV from WeatherPy
cities_df = pd.read_csv("../WeatherPy/Output_Data/cities.csv")

cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bagotville,-28.98,153.42,55.00,87,0,6.93,AU,1595184112
1,Wuwei,37.93,102.64,66.00,54,58,4.21,CN,1595184113
2,Ponta Delgada,37.73,-25.67,71.60,64,75,11.41,PT,1595184115
3,Port Blair,11.67,92.75,81.54,79,100,3.74,IN,1595184116
4,Berlevåg,70.86,29.09,54.99,93,100,9.33,NO,1595184117
...,...,...,...,...,...,...,...,...,...
548,Conner,17.80,121.33,74.16,85,0,2.46,PH,1595184781
549,Coos Bay,43.37,-124.22,75.99,82,1,21.92,US,1595184783
550,Gamba,-2.65,10.00,71.91,85,72,6.73,GA,1595184632
551,Española,35.99,-106.08,84.00,24,1,3.27,US,1595184785


### Humidity Heatmap

In [177]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

# Customize the size of the map
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [155]:
ideal_df = cities_df.loc[(cities_df["Max Temp"] > 60) & (cities_df["Max Temp"] < 90) & \
                         (cities_df["Humidity"] < 40) & (cities_df["Wind Speed"] < 10) & \
                         (cities_df["Cloudiness"] < 50), :]

mapcities_df = ideal_df.dropna()
mapcities_df = mapcities_df.reset_index(drop=True)

### Hotel Map

In [156]:
#Set up hotel dataframe
hotel_df = mapcities_df.loc[:,["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""


In [157]:
#API request to get hotel names
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    location = f"{row['Lat']},{row['Lng']}"
    city = row["City"]

    # add keyword to params dict
    params["location"] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        
    print("------------")


Retrieving Results for Index 0: Neyshabur.
Closest hotel is Amiran Hotel.
------------
Retrieving Results for Index 1: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 2: Camargo.
Closest hotel is Hotel Santa Fe.
------------
Retrieving Results for Index 3: Winnemucca.
Closest hotel is Winners Inn Casino.
------------
Retrieving Results for Index 4: Tursunzoda.
Closest hotel is Stoyanka Charli.
------------
Retrieving Results for Index 5: Price.
Closest hotel is National 9 - Price River Inn.
------------
Retrieving Results for Index 6: Guiratinga.
Closest hotel is Hotel Angra do Cerrado - Guiratinga MT.
------------
Retrieving Results for Index 7: West Wendover.
Closest hotel is Motel 6 Wendover, UT.
------------
Retrieving Results for Index 8: Narsaq.
Closest hotel is Hotel Narsaq.
------------
Retrieving Results for Index 9: Arrondissement de Foix.
Closest hotel is Hôtel TerraNostra - Le Bellevue.
------------
Retrieving Results for Ind

In [158]:
#Erase city with no result
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)

hotel_df

,City,Lat,Lng,Country,Hotel Name
0,Neyshabur,36.21,58.80,IR,Amiran Hotel
1,Mount Isa,-20.73,139.50,AU,ibis Styles Mt Isa Verona
2,Camargo,27.67,-105.17,MX,Hotel Santa Fe
3,Winnemucca,40.97,-117.74,US,Winners Inn Casino
4,Tursunzoda,38.51,68.23,TJ,Stoyanka Charli
5,Price,39.60,-110.81,US,National 9 - Price River Inn
6,Guiratinga,-16.35,-53.76,BR,Hotel Angra do Cerrado - Guiratinga MT
7,West Wendover,40.74,-114.07,US,"Motel 6 Wendover, UT"
8,Narsaq,60.92,-46.05,GL,Hotel Narsaq
9,Arrondissement de Foix,42.83,1.58,FR,Hôtel TerraNostra - Le Bellevue


In [163]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [164]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_locations,
        info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…